In [26]:

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf

import pandas as pd
import numpy as np

from scipy import stats
import librosa
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

to load files in 16000 sample rate

In [ ]:
# def load_wav_16k_mono(filename,i):

#     file_contents = tf.io.read_file(filename)
#     try:
#         wav, sample_rate = tf.audio.decode_wav(file_contents,desired_channels = 1)
#     except Exception as e:
#         subprocess.run(["sox", filename, "-b", "16", "/content/temp_16bit.wav"])
#         file_contents = tf.io.read_file('/content/temp_16bit.wav')
#         wav, sample_rate = tf.audio.decode_wav(file_contents,desired_channels = 1)


#     wav = tf.squeeze(wav,axis = -1)
#     sample_rate = tf.cast(sample_rate, dtype=tf.int64)
#     wav = tfio.audio.resample(wav,rate_in = sample_rate, rate_out = 8000 )
#     wav = wav.numpy()

#     for i in range(len(wav),80000):
#         wav= np.append(wav,0)
#     return wav

In [3]:
TEST_FILE = '/content/drive/MyDrive/ML/datasets/person-identification-using-speech-recognition/training_split/9_23_male/9_23_male_3.wav'
TEST_FILE2 = '/content/drive/MyDrive/ML/datasets/person-identification-using-speech-recognition/training_split/44_22_female/44_22_female_8.wav'


In [28]:
def features_extractor(file,i):
    audio, sample_rate = librosa.load(file)
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)

    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

    return mfccs_scaled_features


In [29]:
train = pd.read_csv('/content/drive/MyDrive/ML/datasets/person-identification-using-speech-recognition/train_mapping.csv')

audio_data = []
for i in range(len(train)):
    path = os.path.join('/content/drive/MyDrive/ML/datasets/person-identification-using-speech-recognition/training_split/',train.iloc[i,0])
    wav = features_extractor(path,i)

    audio_data.append(wav)
    if i % 100 == 0:
      print(i,len(audio_data),len(audio_data[i]))

audio_data = pd.DataFrame(audio_data)


0 1 13
100 101 13
200 201 13
300 301 13
400 401 13
500 501 13
600 601 13
700 701 13
800 801 13
900 901 13
1000 1001 13
1100 1101 13
1200 1201 13
1300 1301 13
1400 1401 13


In [30]:

X_train, X_test, y_train, y_test = train_test_split(audio_data, train['Person_id'], test_size=0.2, random_state=43)
print(len(np.unique(y_train)))


201


---

---

In [31]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=201)
gmm.fit(X_train)

labels = gmm.predict(X_train)

In [32]:
label_map = pd.DataFrame()
label_map['orignal'] = y_train
label_map['predicted'] = labels

In [33]:
label_map.set_index('predicted',inplace=True)
label_map

,orignal
predicted,
30,76
88,155
132,85
30,185
115,85
...,...
70,2
10,7
101,34


In [35]:
label_map.loc[30]

,orignal
predicted,
30,76
30,185
30,196
30,195
30,195
30,76
30,43
30,158
30,195


In [ ]:
def map_labels(test_pred):
  mapped = []
  for item in test_pred:
    label_map


In [ ]:
test_pred = gmm.predict(X_test)
test_pred = map_labels(test_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


rf_classifier = RandomForestClassifier(n_estimators=700, random_state=43)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9037800687285223


----------------------------------------------------


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

num_labels = 201

svm_classifier = SVC(kernel='linear', C=1.0, random_state=43)

svm_classifier.fit(X_train, y_train)

y_pred_svm = svm_classifier.predict(X_test)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("SVM Accuracy:", accuracy_svm)

SVM Accuracy: 0.8350515463917526


---

------

---

In [ ]:
submition = pd.read_csv('/content/drive/MyDrive/ML/datasets/person-identification-using-speech-recognition/Sample_submission - test_mapping.csv')


In [ ]:
audio_data = []
for i in range(len(submition)):
    path = os.path.join('/content/drive/MyDrive/ML/datasets/person-identification-using-speech-recognition/testing_split',str(submition.iloc[i,0])+'.wav')
    wav = features_extractor(path,i)
    audio_data.append(wav)
    if i % 100 ==0:
      print(i)

audio_data = pd.DataFrame(audio_data)

0
100
200
300
400
500
600


In [ ]:
audio_data

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-491.508911,68.242470,-29.810989,4.443399,-17.997978,-17.137157,-19.203506,-4.771843,-4.609623,-1.548820,-12.651922,-10.223746,-10.415616
1,-413.630005,44.931587,-27.756826,1.380349,-19.943609,-27.530609,-23.562885,-6.406269,-4.211088,9.991659,-20.310017,-10.311815,-14.051350
2,-397.793793,39.372189,-30.026936,1.323429,-18.735201,-20.596169,-26.004404,-9.339144,-5.291040,5.671503,-22.828302,-12.655406,-15.435340
3,-436.717865,48.174335,-27.547482,1.960715,-12.076412,-17.604570,-25.146534,-8.991997,-6.871658,5.645524,-22.506374,-16.737761,-8.979218
4,-393.223633,41.510769,-37.635632,-6.775028,-17.592535,-12.909334,-30.058893,-13.976706,-2.638020,1.097564,-15.776166,-13.590219,-8.210648
...,...,...,...,...,...,...,...,...,...,...,...,...,...
632,-378.287201,129.900650,25.671860,23.149363,11.308530,10.333423,-18.988527,-15.148536,-22.843050,-0.861613,-8.152818,-12.811354,-2.967992
633,-410.091675,113.671967,24.964087,14.224825,10.694440,12.394065,-22.849152,4.158904,-18.673080,-7.579477,-7.325856,-9.749763,-2.187011
634,-358.545197,116.832031,22.046717,16.965931,13.070115,10.871352,-27.578932,-1.056156,-20.474003,-9.832223,-7.819269,-10.369866,-4.142944
635,-366.818054,104.691994,28.110283,20.553921,11.634580,11.099681,-27.702417,3.794822,-18.373039,-5.985837,-9.764496,-8.128838,-6.258651


In [ ]:
submition['Person_id'] = rf_classifier.predict(audio_data)

In [ ]:
submition

,File_name,Person_id
0,438,111
1,146,105
2,79,175
3,117,175
4,806,132
...,...,...
632,897,20
633,162,52
634,668,52
635,795,52


In [ ]:
submition.to_csv('/content/submition.csv',index=False)